# Utilization

## Modulation

In [1]:
modules:list = [
    "'pyautogen[retrievechat]' 'qdrant_client[fastembed]' 'flaml[automl]'",
    "langchain-openai==0.0.3",
    "langchain_experimental==0.0.49",
    "huggingface_hub==0.20.2",
    "newspaper3k==0.2.8",
    "wikipedia==1.4.0",
    "moviepy==1.0.3",
    "arxiv==2.1.0",
    "playwright==1.41.1",
    "html2text==2020.1.16",
    "faiss-cpu==1.7.4",
    "-U -q google-generativeai"
]

In [2]:
for module in modules:
  !pip install $module >> "general_module_install_log.txt"

## Packages

In [3]:
import os,shutil,autogen,pathlib,time,uuid,requests,chromadb,faiss,math
import cv2 as cv
import numpy as np
import google.generativeai as genai
import wikipedia as wpp
from chromadb.utils import embedding_functions
from requests.packages.urllib3.exceptions import InsecureRequestWarning,InsecurePlatformWarning
from typing import Union,Optional,Any,Type
from datetime import datetime,timedelta
from PIL import Image as PILImage
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import HuggingFaceHub
from langchain.tools import BaseTool,WikipediaQueryRun
from langchain.pydantic_v1 import BaseModel,Field
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_transformers import Html2TextTransformer
from langchain.document_loaders import AsyncChromiumLoader
from langchain_community.vectorstores import FAISS
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.utilities.arxiv import ArxivAPIWrapper
from langchain_community.document_loaders import NewsURLLoader
from langchain_experimental.generative_agents import GenerativeAgent,GenerativeAgentMemory
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain_openai import OpenAI as LOpenAI
from autogen.agentchat.contrib.retrieve_assistant_agent import RetrieveAssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent
from autogen.agentchat.contrib.qdrant_retrieve_user_proxy_agent import QdrantRetrieveUserProxyAgent
from autogen.code_utils import content_str
from qdrant_client import QdrantClient
from google.colab import userdata

In [4]:
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
requests.packages.urllib3.disable_warnings(InsecurePlatformWarning)

In [5]:
import warnings
warningList:list = [DeprecationWarning,FutureWarning,UserWarning,RuntimeWarning]
for warn in warningList: warnings.filterwarnings("ignore",category=warn)

## Class

In [6]:
class ClassInitial(object): pass
class ErrorInitial(Exception): pass
class NullInitial(object): pass
class ProcessInitial(object): pass
class ModelInitial(object): pass
class ResultInitial(object): pass
class FunctionInitial(object): pass

In [7]:
class File:
  def __init__(self,path:str,name:str=None)->Optional[ClassInitial]:
    self.path = path
    if name:
      self.name = name
    else:
      self.name = str(uuid.uuid4()).replace("-","_")
    self.source = self.GetSource(self.path)
  def FileResponse(self,response:Union[str,ClassInitial])->ProcessInitial:
    self.response = response
  def GetSource(self,path:str,prefix:str="_")->str:
    parts = path.split(prefix)
    return parts[0]

In [8]:
class Constants:
  imageDirectory = os.path.join(os.getcwd(),"image_directory")
  wikipediaDirectory = os.path.join(os.getcwd(),"wikipedia_directory")
  arxivDirectory = os.path.join(os.getcwd(),"arxiv_directory")
  newsDirectory = os.path.join(os.getcwd(),"news_directory")
  googleDirectory = os.path.join(os.getcwd(),"google_directory")
  lastResponseFile = os.path.join(os.getcwd(),"last_response.txt")
  googleResponseFile = os.path.join(os.getcwd(),"google_response.txt")
  wikipediaResponseFile = os.path.join(os.getcwd(),"wikipedia_response.txt")
  arxivResponseFile = os.path.join(os.getcwd(),"arxiv_response.txt")
  newsResponseFile = os.path.join(os.getcwd(),"news_response.txt")
  htmlResponseFile = os.path.join(os.getcwd(),"html_response.txt")
  geminiResponseFile = os.path.join(os.getcwd(),"gemini_response.txt")
  historyResponseFile = os.path.join(os.getcwd(),"history_response.txt")

## Functions

In [175]:
CreateDirectory:Optional[FunctionInitial] = lambda path:os.mkdir(path) if not os.path.exists(path) else None
DeleteDirectory:Optional[FunctionInitial] = lambda path:shutil.rmtree(path) if len(os.listdir(path)) > 0 else None
KeyControl:Optional[FunctionInitial] = lambda key:key.name.lower().replace(" ","_") if " " in key.name else key.name
DeleteHTTP:Optional[FunctionInitial] = lambda site: site.replace("http://","").replace("https://","").rstrip().lstrip()
GlobalChecker:Optional[FunctionInitial] = lambda variable: str(variable) in globals()

In [176]:
def PrintDecoder(func:ClassInitial)->Union[str,None]:
  def Wrapper(*args,**kwargs)->Union[str,None]:
    result = func(*args,**kwargs)
    print(f"[::FROM SYSTEM ::] {str(result).upper()}")
    return result
  return Wrapper

In [177]:
def TimerDecoder(func:ClassInitial)->Union[str,None]:
  def Wrapper(*args,**kwargs)->Union[str,None]:
    before = time.time()
    result = func(*args,**kwargs)
    after = time.time()
    difference = after-before
    print(f"[:: FROM SYSTEM ::] FUNCTION: {str(func.__name__)} EXECUTED IN {difference:.5f} SECONDS")
    return result
  return Wrapper

In [178]:
def IsTerminated(message:Union[dict,ClassInitial])->bool:
  if isinstance(message,dict):
    message = message.get("content")
    if message is None:
      return False
    else:
      return message.rstrip().endswith("TERMINATE")

In [179]:
def StrToList(message:str)->list:
  if message is not None:
    output = message.strip("][").replace("'","")
    output = output.strip('][').replace('"','').replace("]","").replace("[","").split(", ")
    return output
  else:
    return []

In [180]:
@TimerDecoder
def SaveImages(images:Union[str,list])->ProcessInitial:
  CreateDirectory(Constants.imageDirectory)
  if isinstance(images,str): images = [images]
  for count,image in enumerate(images):
    try:
      if ("https://" in str(image)) or ("http://" in str(image)):
        if ("jpg" in image) or ("png" in image) or ("jpeg" in image) or ("JPG" in image):
          raw = requests.get(image,verify=False,stream=True,allow_redirects=True,timeout=30).raw
          raw = PILImage.open(raw)
          raw = np.array(raw,dtype=np.uint8)
          path = os.path.join(os.getcwd(),Constants.imageDirectory,f"saved_image_{count}.jpg")
          cv.imwrite(path,raw)
        else:
          pass
      else:
        path = os.path.join(os.getcwd(),Constants.imageDirectory,f"saved_image_{count}.jpg")
        image = cv.imread(str(image))
        image = np.array(image,dtype=np.uint8)
        cv.imwrite(path,image)
    except:
      pass

In [181]:
class ErrorModule(ErrorInitial):
  def __init__(self,errorType:ErrorInitial=NotImplementedError)->Optional[ClassInitial]:
    self.error = errorType
  def __str__(self)->str:
    return "ERROR MODULATION"
  def __call__(self,message:Optional[str]=NotImplemented)->ErrorInitial:
    raise self.error(message)
  def __len__(self)->int:
    return 0
  def __getstate__(self,message:Optional[str]=NotImplemented)->ErrorInitial:
    return self.error(message)
  def __repr__(self)->str:
    return ErrorModule.__doc__
  @property
  def Default(self,message:Optional[str]=NotImplemented)->ErrorInitial:
    raise self.error(message)
  def Manuel(self,error:ErrorInitial,message:Optional[str])->ErrorInitial:
    raise error(message)

In [182]:
errorEngine = ErrorModule()

In [183]:
class WriteOutput(FunctionInitial):
  def __init__(self,path:Union[str,None]=None)->Optional[ClassInitial]:
    if path is None:
      self.__path = Constants.lastResponseFile
    else:
      self.__path = path
    self.ops = None
  def __str__(self)->str:
    return "WRITE TXT MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return WriteOutput.__doc__
  def __len__(self)->int:
    return 0
  def Text(self,text:Union[str,bytes])->ProcessInitial:
    if isinstance(text,bytes): text = text.decode("utf-8")
    return self.ops.write(str(text))
  def __enter__(self)->ProcessInitial:
    self.ops = open(self.__path,"w",encoding="utf-8")
    return self
  def __exit__(self,ex:ClassInitial,et:ClassInitial,eb:ClassInitial)->ProcessInitial:
    self.ops.close()

In [184]:
class Credentials(FunctionInitial):
  def __init__(self)->Optional[ClassInitial]:
    self.__google = userdata.get("GOOGLE_API_KEY")
    self.__gemini = userdata.get("GEMINI_NEW_API")
    self.__hugging = userdata.get("HUGGINGFACEHUB_API_TOKEN")
    self.__openai = userdata.get("OPENAI_API_KEY")
    self.__cse = userdata.get("CSE_ID_KEY")
  def __str__(self)->str:
    return "CREDENTIALS MODULATION"
  @TimerDecoder
  def __call__(self)->ProcessInitial:
    os.environ["OPENAI_API_KEY"] = self.__openai
    os.environ["GEMINI_API_KEY"] = self.__gemini
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = self.__hugging
    os.environ["GOOGLE_CSE_ID"] = self.__cse
    os.environ["GOOGLE_API_KEY"] = self.__google
    self.HuggingLogin()
    print("[:: FROM SYSTEM ::] CREDENTIALS HAS BEEN DEFINED")
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return Credentials.__doc__
  def __len__(self)->int:
    return 0
  def HuggingLogin(self)->ProcessInitial:
    if "HUGGINGFACEHUB_API_TOKEN" in os.environ:
      globals()["HUGGING_LOGIN_KEY"] = self.__hugging
      !huggingface-cli login --token $HUGGING_LOGIN_KEY
    else:
      os.environ["HUGGINGFACEHUB_API_TOKEN"] = self.__hugging
      globals()["HUGGING_LOGIN_KEY"] = self.__hugging
      !huggingface-cli login --token $HUGGING_LOGIN_KEY

In [185]:
credentials = Credentials()
credentials()

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful
[:: FROM SYSTEM ::] CREDENTIALS HAS BEEN DEFINED
[:: FROM SYSTEM ::] FUNCTION: __call__ EXECUTED IN 0.70760 SECONDS


## Requirements

In [186]:
safety:list = [
    {
        "category":"HARM_CATEGORY_HARASSMENT",
        "threshold":"BLOCK_NONE"
    },
    {
        "category":"HARM_CATEGORY_HATE_SPEECH",
        "threshold":"BLOCK_NONE"
    },
    {
        "category":"HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold":"BLOCK_NONE"
    },
    {
        "category":"HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold":"BLOCK_NONE"
    }
]

In [187]:
parameters:dict = {
    "temperature":0.2,
    "top_k":1,
    "max_output_tokens":4096,
    "candidate_count":1
}

In [188]:
configuration:list = [
    {
        "model":"gpt-4-1106-preview",
        "api_key":os.environ.get("OPENAI_API_KEY")
    },
    {
        "model":"gpt-4-vision-preview",
        "api_key":os.environ.get("OPENAI_API_KEY")
    },
    {
        "model":"dalle",
        "api_key":os.environ.get("OPENAI_API_KEY")
    },
    {
        "model": "gemini-pro",
        "api_key": os.environ.get("GEMINI_API_KEY")
    },
    {
        "model": "gemini-pro-vision",
        "api_key": os.environ.get("GEMINI_API_KEY")
    },
    {
        "model": "gemini-1.5-pro-latest",
        "api_key": os.environ.get("GEMINI_API_KEY")
    }
]

# Searching Modulation

## Google

In [189]:
class GoogleSearch(object):
  def __init__(self,similarity:int=1,power:int=1)->Optional[ClassInitial]:
    self.similarity = int(similarity)
    self.power = int(power)
    self.wrapper = GoogleSearchAPIWrapper(k=self.similarity,siterestrict=False)
  def __str__(self)->str:
    return "GOOGLE SEARCH MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __len__(self)->int:
    return 0
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return GoogleSearch.__doc__
  @TimerDecoder
  def Query(self,topic:str,sites:Union[str,list]=None)->tuple:
    content = ""
    links = []
    if sites is not None:
      queryList = []
      if isinstance(sites,str): sites=[sites]
      for site in sites:
        query = f"site: {DeleteHTTP(str(site).lower()).lstrip().rstrip()} '{topic}'"
        queryList.append(query)
      for query in queryList:
        result = self.wrapper.results(query,num_results=self.power)
        if (result is not None) and (len(result) > 0):
          try:
            for idx in range(len(result)):
              data = result[idx]
              content += (
                  f"TITLE: {data['title']}\nDESCRIPTION: {data['snippet']}\n\n"
              )
              links.append(data['link'])
          except:
            pass
        else:
          pass
        return content,links
    else:
      result = self.wrapper.results(query=str(topic),num_results=self.power)
      if (result is not None) and (len(result) > 0):
        try:
          for idx in range(len(result)):
            data = result[idx]
            content += (
                f"TITLE: {data['title']}\nDESCRIPTION: {data['snippet']}\n\n"
            )
            links.append(data['link'])
        except:
          pass
      else:
        pass
      if content:
        with WriteOutput(Constants.googleResponseFile) as ops:
          ops.Text(str(content))
      return content,links

## Wikipedia

In [190]:
class WikipediaSearch(object):
  def __init__(self,power:int=1)->Optional[ClassInitial]:
    self.power = int(power)
  def __str__(self)->str:
    return "WIKIPEDIA SEARCH MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __len__(self)->int:
    return 0
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return WikipediaSearch.__doc__
  @TimerDecoder
  def Query(self,topic:str,**load_kwargs:Any)->tuple:
    content = ""
    links = []
    result = wpp.search(str(topic),results=self.power)
    if (result is not None) and (isinstance(result,list) and len(result) > 0):
      for idx in range(len(result)):
        try:
          target = result[idx]
          base = wpp.page(str(target),**load_kwargs)
          try:
            images = base.images
            if (isinstance(images,list) and len(images) > 0):
              SaveImages(images)
          except:
            pass
          content += f"TITLE: {base.title}\n"
          try:
            content += f"DESCRIPTION: {base.summary}\n\n"
          except:
            content += f"DESCRIPTION: {base.content}\n\n"
          links.append(base.url)
        except:
          pass
    else:
      pass
    print(f"[:: FROM SYSTEM ::] {len(images)} IMAGES HAVE BEEN SAVED TO {Constants.imageDirectory}")
    if content:
      with WriteOutput(Constants.wikipediaResponseFile) as ops:
        ops.Text(str(content))
    return content,links

## News

In [191]:
class NewsSearch(object):
  def __init__(self,isNLP:bool=True)->Optional[ClassInitial]:
    self.nlp = isNLP
    self.optimization = dict(
        text_mode=True,
        continue_on_failure=True,
        show_progress_bar=False
    )
    if self.nlp:
      try:
        import nltk
        nltk.download("punkt")
        self.optimization["nlp"] = True
      except:
        self.optimization["nlp"] = False
    else:
      self.optimization["nlp"] = False
  def __str__(self)->str:
    return "NEWS SEARCH MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __len__(self)->int:
    return 0
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return NewsSearch.__doc__
  @TimerDecoder
  def Query(self,urls:Union[str,list])->str:
    content = ""
    if isinstance(urls,str): urls = [urls]
    self.optimization["urls"] = urls
    loader = NewsURLLoader(**self.optimization)
    result = loader.load()
    if (result is not None) and (len(result) > 0 ):
      try:
        for idx in range(len(result)):
          data = result[idx]
          if data.metadata["description"]:
            content += f"{idx+1} NEWS TEXT:\n"
            content += data.metadata["description"]+"\n\n"
          else:
            content += f"{idx+1} NEWS TEXT:\n"
            content += data.metadata["summary"]+"\n\n"
      except:
        pass
    else:
      pass
    if content:
      with WriteOutput(Constants.newsResponseFile) as ops:
        ops.Text(str(content))
    return content


## Arxiv

In [192]:
class ArxivSearch(object):
  def __init__(self,similarity:int=1,power:int=1)->Optional[ClassInitial]:
    self.similarity = int(similarity)
    self.power = int(power)
    self.wrapper = ArxivAPIWrapper(
        top_k_results=self.power,
        load_max_docs=self.similarity,
        load_all_available_meta=True,
        doc_content_chars_max=10**4,
        ARXIV_MAX_QUERY_LENGTH=500
    )
  def __str__(self)->str:
    return "ARXIV SEARCH MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return ArxivSearch.__doc__
  def __len__(self)->int:
    return 0
  @TimerDecoder
  def Query(self,topic:str)->str:
    content = ""
    result = self.wrapper.get_summaries_as_docs(str(topic))
    if (result is not None) and (isinstance(result,list) and len(result) > 0):
      for idx in range(len(result)):
        try:
          data = result[idx]
          content += (
              f"TITLE: {data.metadata['Title']}\n"
              f"AUTHORS: {data.metadata['Authors']}\n"
              f"DESCRIPTION: {data.page_content}\n\n"
          )
        except:
          pass
    else:
      pass
    if content:
      with WriteOutput(Constants.arxivResponseFile) as ops:
        ops.Text(str(content))
    return content


## HTML

In [193]:
class HTMLSearch(object):
  def __init__(self)->Optional[ClassInitial]:
    try:
      import nest_asyncio
      nest_asyncio.apply()
      !playwright install >> "playwright_log.txt"
    except:
      pass
    self.__engine = Html2TextTransformer(ignore_images=False,ignore_links=False)
  def __str__(self)->str:
    return "HTML TO TEXT MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return HTMLSearch.__doc__
  def __len__(self)->str:
    return HTMLSearch.__doc__
  def Query(self,urls:Union[str,list])->str:
    content = ""
    links = []
    if isinstance(urls,str): urls = [urls]
    loader = AsyncChromiumLoader(urls)
    document = loader.load()
    if document is not None:
      transform = self.__engine.transform_documents(document)
      if (isinstance(transform,list) and len(transform) > 0) or (isinstance(transform,tuple) and len(transform) > 0):
        for idx in range(len(transform)):
          try:
            data = transform[idx]
            content += (
                f"{idx+1} PAGE:\n"
                f"CONTENT: {data.page_content}"
            )
            try:
              links.append(data.metadata["source"])
            except:
              pass
          except:
            pass
      else:
        pass
    else:
      pass
    if content:
      with WriteOutput(Constants.htmlResponseFile) as ops:
        ops.Text(str(content))
    return content,links

# Retriever Process

## Splitter

In [194]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    separators=["\n","\r","\t"]
)

## Gemini Structure

In [195]:
class GeminiModel(ModelInitial):
  def __init__(self)->Optional[ClassInitial]:
    genai.configure(api_key=userdata.get("GEMINI_NEW_API"))
    self.__imagePath = Constants.imageDirectory
    self.__gemini = genai.GenerativeModel(
        model_name="models/gemini-1.5-pro-latest",
        generation_config=parameters,
        safety_settings=safety
    )
    self.__prompt = (
        "You are an expert personality trait analyst for {person_target}. "
        "You are given information from different sources and you reveal characteristics about the target person based on this source information. "
        "You must create these personality traits very specifically and realistically. "
        "You will be given visuals about the subject, evaluate these when determining personality traits. "
        "Return them as a string array without any comment, like this: ['anti-war', 'intellectual', 'detailed thinker']\n"
        "Below is the actual data for you to use:\n"
        "From Wikipedia:\n{wikipedia_information}\n\n"
        "From Google:\n{google_information}\n\n"
        "From News:\n{news_information}\n\n"
        "From Arxiv:\n{arxiv_information}\n\n"
        "From HTML Site:\n{html_information}\n\n"
        "Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI."
    )
    self.__promptChat = (
        "You are an expert character information extractor for {person_target}. "
        "You are given information from different sources and you reveal characteristics about the target person. "
        "You must extract character informations very specifically and realistically. "
        "You will be given visuals about the subject, evaluate these when determining character informations. "
        "Use one-sentence descriptive phrases. "
        "Make complete sentences in accordance with the language rules. "
        "Return them as a string array without any comment, like this: ['He/she remembers his/her dog, from when he/she was a kid', 'He/she feels tired from driving so far', 'He/she sees the new home']\n"
        "Below is the actual data for you to use:\n"
        "From Wikipedia:\n{wikipedia_information}\n\n"
        "From Google:\n{google_information}\n\n"
        "From News:\n{news_information}\n\n"
        "From Arxiv:\n{arxiv_information}\n\n"
        "From HTML Site:\n{html_information}\n\n"
        "Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI."
    )
    self.__chat = self.__gemini.start_chat(history=[])
    self.lenControl = False
  def __str__(self)->str:
    return "GEMINI MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return GeminiModel.__doc__
  def __len__(self)->int:
    return 0
  def GetDirectory(self)->list:
    images = []
    if os.path.exists(self.__imagePath):
      for base,_,files in os.walk(self.__imagePath):
        try:
          for f in files: images.append(os.path.join(base,f))
        except:
          pass
    return images
  def GetUpload(self)->ProcessInitial:
    images = self.GetDirectory()
    uploaded = []
    try:
      for idx in images: uploaded.append(File(path=idx))
    except:
      pass
    return uploaded
  def GetFiles(self)->ProcessInitial:
    uploaded = self.GetUpload()
    files = []
    for idx in uploaded:
      try:
        response = genai.upload_file(path=idx.path)
        idx.FileResponse(response)
        files.append(idx)
      except:
        pass
    print(f"[:: FROM SYSTEM ::] MODEL FILES HAVE BEEN UPLOADED - TOTAL: {len(files)}")
    return files
  def GetRequest(self,person:str,wikipediaContent:str,googleContent:str,newsContent:str,arxivContent:str,htmlContent:str)->Union[tuple,ProcessInitial]:
    self.__prompt = self.__prompt.format(
        person_target=str(person).upper(),
        wikipedia_information=str(wikipediaContent),
        google_information=str(googleContent),
        news_information=str(newsContent),
        arxiv_information=str(arxivContent),
        html_information=str(htmlContent)
    )
    modelRequest = [self.__prompt]
    files = self.GetFiles()
    for idx in files:
      modelRequest.append(idx.response)
      modelRequest.append(idx.source)
    return modelRequest,files
  @TimerDecoder
  def Query(self,person:str,wikipediaContent:str,googleContent:str,newsContent:str,arxivContent:str,htmlContent:str)->Union[str,ProcessInitial]:
    self.lenControl = True if len(os.listdir(self.__imagePath)) else False
    self.__promptChat = self.__promptChat.format(
        person_target=str(person).upper(),
        wikipedia_information=str(wikipediaContent),
        google_information=str(googleContent),
        news_information=str(newsContent),
        arxiv_information=str(arxivContent),
        html_information=str(htmlContent)
    )
    if self.lenControl:
      modelRequest,files = self.GetRequest(person,wikipediaContent,googleContent,newsContent,arxivContent,htmlContent)
      response = self.__gemini.generate_content(
          modelRequest,
          request_options={"timeout":600}
      )
      if response is not None:
        try:
          response.resolve()
          if len(response.parts) > 0:
            answer = response.text
            historyCharacter = self.__chat.send_message(self.__promptChat)
            historyCharacter = historyCharacter.text
          else:
            answer = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
            historyCharacter = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
        except:
          if len(response.parts) > 0:
            answer = response.text
            historyCharacter = self.__chat.send_message(self.__promptChat)
            historyCharacter = historyCharacter.text
          else:
            answer = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
            historyCharacter = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
        try:
          for idx in files:
            genai.delete_file(idx.response.name)
          print(f"[:: FROM SYSTEM ::] MODEL FILES HAVE BEEN DELETED AFTER PROCESS- TOTAL: {len(files)}")
        except:
          pass
      else:
        answer = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
        historyCharacter = "[:: FROM MODEL ::] NO RESPONSE FROM MODEL"
    else:
      answer = "[:: FROM SYSTEM ::] NO FILES TO PROCESS - CHECK YOUR SYSTEM OR RESET NOTEBOOK"
      historyCharacter = "[:: FROM SYSTEM ::] NO FILES TO PROCESS - CHECK YOUR SYSTEM OR RESET NOTEBOOK"
    if (answer) and ("[:: FROM MODEL ::]" not in answer and "[:: FROM SYSTEM ::]" not in answer):
      with WriteOutput(Constants.geminiResponseFile) as ops:
        ops.Text(str(answer))
    if (historyCharacter) and ("[:: FROM MODEL ::]" not in historyCharacter and "[:: FROM SYSTEM ::]" not in historyCharacter):
      with WriteOutput(Constants.historyResponseFile) as ops:
        ops.Text(str(historyCharacter))
    return answer,historyCharacter


- **TEST OPTION:**

```
cntG,lnkG = GoogleSearch(4).Query("Nikola Tesla")
cntW,lnkW = WikipediaSearch(4).Query("Nikola Tesla")
cntH,lnkH = HTMLSearch().Query(lnkG)
cntA = ArxivSearch().Query("Nikola Tesla")
lnkG.extend(lnkW)
cntN = NewsSearch().Query(lnkG)


geminiBase = GeminiModel()

answer,history = geminiBase.Query(
    person="Nikola Tesla",
    wikipediaContent=cntW,
    googleContent=cntG,
    newsContent=cntN,
    arxivContent=cntA,
    htmlContent=cntH
)

print(history)

print(answer)
```

## RAG Structure

In [196]:
class RagModel(ModelInitial):
  def __init__(self)->ClassInitial:
    self.__model = "gemini-pro"
    self.__prompt = (
        "You are now {person_target}. "
        "Your answers must be realistic, sincere and appropriate to the personality in the documents given to you. "
        "Observe yourself and always be honest. "
        "Respond to orders and provide detailed information within the scope of the information and files given to you. "
        "Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI. "
    )
    self.__autoReply = "Always give the final answer as text and add 'TERMINATE' at the end."
    self.proxyOptimization = dict(
        name="ragproxyagent",
        human_input_mode="NEVER",
        default_auto_reply=self.__autoReply,
        max_consecutive_auto_reply=15,
        retrieve_config={
            "task":"qa",
            "docs_path":[],
            "chunk_token_size":2000,
            "model":self.__model,
            "client":QdrantClient(":memory:"),
            "embedding_model":"all-mpnet-base-v2",
            "get_or_create":True,
            "custom_text_types":autogen.retrieve_utils.TEXT_FORMATS,
            "custom_text_split_function":splitter.split_text
        },
        code_execution_config=False
    )
  def __str__(self)->str:
    return "RAG MODEL MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __len__(self)->int:
    return 0
  def __repr__(self)->str:
    return RagModel.__doc__
  def RetrieverAssistant(self,person:str)->ModelInitial:
    self.__prompt = self.__prompt.format(person_target=str(person))
    assistant = RetrieveAssistantAgent(
        name="assistant",
        system_message=self.__prompt,
        llm_config={
            "timeout":600,
            "cache_seed":42,
            "config_list":configuration
        }
    )
    return assistant
  def RetrieverProxy(self,documents:Union[str,list])->ModelInitial:
    if isinstance(documents,str): documents = [documents]
    if len(documents) > 0:
      for doc in documents:
        self.proxyOptimization["retrieve_config"]["docs_path"].append(str(doc))
    proxy = QdrantRetrieveUserProxyAgent(**self.proxyOptimization)
    return proxy

- **TEST OPTION:**

```
docsList = [
    "/content/arxiv_response.txt",
    "/content/google_response.txt",
    "/content/news_response.txt",
    "/content/wikipedia_response.txt",
    "/content/history_response.txt",
    "/content/gemini_response.txt",
    "/content/html_response.txt"
]

rag = RagModel()
assistant = rag.RetrieverAssistant(person="Nikola Tesla")
proxy = rag.RetrieverProxy(documents=docsList)
question = "Who are you?"
result = proxy.initiate_chat(assistant,message=proxy.message_generator,problem=question)
```

# Personality Reflection Process

## Relevance Score

In [197]:
def RelevanceScore(score:Union[int,float])->float:
  # This function converts the euclidean norm of normalized embeddings
  # (0 is most similar, sqrt(2) most dissimilar) to a similarity function (0 to 1)
  return 1.0-score/math.sqrt(2)

## Embedding Function

In [198]:
embedding = embedding_functions.OpenAIEmbeddingFunction(
    api_key=userdata.get("OPENAI_API_KEY"),
    model_name="text-embedding-ada-002"
)

## Memory Function

In [199]:
def MemoryRetriever()->Optional[ClassInitial]:
  embeddingModel = OpenAIEmbeddings(model="text-embedding-ada-002")
  size = 1536
  index = faiss.IndexFlatL2(size)
  store = FAISS(
      embeddingModel.embed_query,
      index,
      InMemoryDocstore({}),
      {},
      relevance_score_fn=RelevanceScore
  )
  vector = TimeWeightedVectorStoreRetriever(
      vectorstore=store,
      other_score_keys=["importance"],
      k=15
  )
  return vector

## LLM Structure

In [200]:
class LLMStructure(ModelInitial):
  def __init__(self)->Optional[ClassInitial]:
    self.modelMistral = "mistralai/Mistral-7B-Instruct-v0.1"
    self.modelOpenAI = "gpt-4"
  def __str__(self)->str:
    return "LLM STRUCTURE MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __len__(self)->int:
    return 0
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __repr__(self)->str:
    return LLMStructure.__doc__
  def Load(self,llmType:str="openai")->ModelInitial:
    if llmType.lower() == "openai":
      llm = ChatOpenAI(
          temperature=0.2,
          max_tokens=4096,
          model_name=self.modelOpenAI,
          api_key=userdata.get("OPENAI_API_KEY")
      )
      return llm
    elif llmType.lower() == "hugging":
      llm = HuggingFaceHub(
          repo_id=self.modelMistral,
          task="text-generation",
          model_kwargs={"temperature":0.2}
      )
      return llm
    else:
      errorEngine.Manuel(ValueError,"[:: FROM SYSTEM ::] NOT A VIABLE LLM MODEL")

In [201]:
openAIModel = LLMStructure().Load()

## Personality Functions

In [202]:
def GenerativePersonality(traits:list,history:list,name:str,age:int,status:str,llm:ModelInitial=openAIModel,reflection:int=8)->Union[ModelInitial,ProcessInitial]:
  traits = ", ".join(traits) if len(traits) > 0 else "N/A"
  memory = GenerativeAgentMemory(
      llm=llm,
      memory_retriever=MemoryRetriever(),
      verbose=False,
      reflection_threshold=int(reflection)
  )
  person = GenerativeAgent(
      name=str(name),
      age=int(age),
      traits=traits,
      status=str(status),
      memory_retriever=MemoryRetriever(),
      llm=llm,
      memory=memory,
      summary_refresh_seconds=3600
  )
  if len(history) > 0:
    for idx in history:
      person.memory.add_memory(str(idx))
  else:
    pass
  return person


## Callback

In [203]:
def PersonCallback(message:Optional[str],person:ModelInitial,name:str)->Union[None,str]:
  response = person.generate_dialogue_response(str(message))
  if (isinstance(response,list) and len(response) > 0) or (isinstance(response,tuple) and len(response) > 0):
    output = response[0]
    if output:
      data = response[1]
      data = data.replace(f"{name} said","").replace(f"{name} said ","")
      return data
    else:
      return None
  else:
    return None

## Action

In [204]:
def PersonAction(traits:list,history:list,message:str,name:str,age:int,status:str="N/A",llm:ModelInitial=openAIModel,reflection:int=8)->Union[str,ProcessInitial]:
  person = GenerativePersonality(
      traits=traits,
      history=history,
      name=str(name),
      age=int(age),
      status=str(status),
      llm=llm,
      reflection=int(reflection)
  )
  try:
    summary = person.get_summary(force_refresh=True)
  except:
    pass
  response = PersonCallback(message,person,name)
  if (response is not None) and (response != " "):
    return str(response)
  else:
    return None

# Main Operation

## Module

In [205]:
class MainOperation(object):
  def __init__(self)->Optional[ClassInitial]:
    self.__rag = RagModel()
    self.__google = GoogleSearch()
    self.__wikipedia = WikipediaSearch()
    self.__arxiv = ArxivSearch()
    self.__news = NewsSearch()
    self.__html = HTMLSearch()
    self.__gemini = GeminiModel()
    self.googleResponseDirectory = Constants.googleResponseFile
    self.wikipediaResponseDirectory = Constants.wikipediaResponseFile
    self.arxivResponseDirectory = Constants.arxivResponseFile
    self.newsResponseDirectory = Constants.newsResponseFile
    self.htmlResponseDirectory = Constants.htmlResponseFile
    self.geminiResponseDirectory = Constants.geminiResponseFile
    self.historyResponseDirectory = Constants.historyResponseFile
    self.imageResponseDirectory = Constants.imageDirectory
  def __str__(self)->str:
    return "MAIN OPERATION MODULATION"
  def __call__(self)->NullInitial:
    return None
  def __getstate__(self)->ErrorInitial:
    errorEngine()
  def __len__(self)->int:
    return 0
  def __repr__(self)->str:
    return MainOperation.__doc__
  def DocumentDirectoryCheck(self)->list:
    documents = []
    if os.path.exists(self.googleResponseDirectory): documents.append(self.googleResponseDirectory)
    if os.path.exists(self.wikipediaResponseDirectory): documents.append(self.wikipediaResponseDirectory)
    if os.path.exists(self.arxivResponseDirectory): documents.append(self.arxivResponseDirectory)
    if os.path.exists(self.newsResponseDirectory): documents.append(self.newsResponseDirectory)
    if os.path.exists(self.htmlResponseDirectory): documents.append(self.htmlResponseDirectory)
    if os.path.exists(self.geminiResponseDirectory): documents.append(self.geminiResponseDirectory)
    if os.path.exists(self.historyResponseDirectory): documents.append(self.historyResponseDirectory)
    return documents
  @TimerDecoder
  def Launch(self,person:str,question:str,age:int,isRag:bool=False,status:str="N/A",llm:ModelInitial=openAIModel,reflection:int=8,power:int=None)->Union[str,ProcessInitial]:
    content = ""
    assistantRAG = self.__rag.RetrieverAssistant(person=str(person))
    if power is not None:
      self.__google.power = int(power)
      self.__wikipedia.power = int(power)
      self.__arxiv.power = int(power)
    else:
      pass
    contentArxiv = self.__arxiv.Query(str(person))
    print(f"[:: FROM SYSTEM ::] ARXIV GATHERING - DONE : CONTENT LENGTH: {len(contentArxiv)} --> NEXT: GOOGLE")
    contentGoogle,linkGoogle = self.__google.Query(str(person))
    print(f"[:: FROM SYSTEM ::] GOOGLE GATHERING - DONE : CONTENT LENGTH: {len(contentGoogle)} --> NEXT: WIKIPEDIA")
    contentWikipedia,linkWikipedia = self.__wikipedia.Query(str(person))
    print(f"[:: FROM SYSTEM ::] WIKIPEDIA GATHERING - DONE : CONTENT LENGTH: {len(contentWikipedia)} --> NEXT: HTML CONTENT")
    contentHTML,linkHTML = self.__html.Query(linkGoogle)
    print(f"[:: FROM SYSTEM ::] HTML TEXT GATHERING - DONE : CONTENT LENGTH: {len(contentHTML)} --> NEXT: NEWS CONTENT")
    linkGoogle.extend(linkWikipedia)
    contentNews = self.__news.Query(linkGoogle)
    print(f"[:: FROM SYSTEM ::] NEWS GATHERING - DONE : CONTENT LENGTH: {len(contentNews)} --> NEXT: GEMINI RESPONSE")
    traitGemini,historyGemini = self.__gemini.Query(
        person=str(person),
        wikipediaContent=contentWikipedia,
        googleContent=contentGoogle,
        newsContent=contentNews,
        arxivContent=contentArxiv,
        htmlContent=contentHTML
    )
    print("[:: FROM SYSTEM ::] TRAIT & HISTORY GATHERING - DONE  --> NEXT: PERSON RESPONSE")
    try:
      traitList = StrToList(traitGemini)
      historyList = StrToList(historyGemini)
    except:
      traitList = []
      historyList = []
    print(f"[:: FROM SYSTEM ::] TRAIT LENGTH: {len(traitList)}")
    print(f"[:: FROM SYSTEM ::] HISTORY LENGTH: {len(historyList)}")
    if isRag:
      print("[:: FROM SYSTEM ::] RAG IS ACTIVE")
      documents = self.DocumentDirectoryCheck()
      proxyRAG = self.__rag.RetrieverProxy(documents=documents)
      resultRAG = proxyRAG.initiate_chat(
          assistantRAG,
          message=proxyRAG.message_generator,
          problem=str(question)
      )
      try:
        if len(resultRAG.chat_history) > 0:
          contentRAG = resultRAG.chat_history[-1]["content"]
          print(f"[:: FROM SYSTEM ::] RAG - DONE : CONTENT LENGTH: {len(contentRAG)}")
        else:
          contentRAG = None
      except:
        contentRAG = None
    else:
      print("[:: FROM SYSTEM ::] RAG IS INACTIVE")
      contentRAG = None
    personReflection = PersonAction(
        traits=traitList,
        history=historyList,
        message=str(question),
        name=str(person),
        age=int(age),
        status=str(status),
        llm=llm,
        reflection=int(reflection)
    )
    print(f"[:: FROM SYSTEM ::] PERSON REFLECTION - DONE : CONTENT LENGTH: {len(personReflection)}")
    if personReflection is not None:
      content += f"PERSON REFLECTION AGENT ANSWER:\n{str(personReflection)}\n\n"
    if contentRAG is not None:
      content += f"RETRIEVEL AGENT ANSWER:\n{str(contentRAG)}\n\n"
    return content

In [206]:
ops = MainOperation()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


- **TEST OPTION:**

```
ops = MainOperation()

content = ops.Launch(
    person="Nikola Tesla",
    question="Who are you?",
    age=30,
    isRag=False
)
```

# Tool Configuration

## Inputs

In [207]:
class OperationInput(BaseModel):
  target:Optional[str] = Field(description="User input for target person to create")
  question:Optional[str] = Field(description="User question to person")
  age:Optional[int] = Field(description="Age set for the created person")
  power:Optional[int] = Field(description="Search power on the internet for target person")

## Structure

In [208]:
class OperationTool(BaseTool):
  name = "Person_Creation_Resuscitation_Tool"
  description = "Use this tool to bring the person back to life and simulate."
  args_schema = OperationInput
  def _run(self,target:Optional[str],question:Optional[str],age:Optional[int],power:Optional[int]=1)->Union[str,ProcessInitial]:
    content = ops.Launch(
        person=str(target),
        question=str(question),
        age=int(age),
        power=int(power),
        isRag=False
    )
    return content


In [209]:
operationTool = OperationTool()

## Functions

In [210]:
def ToolDefine(baseTool:Union[ClassInitial,BaseTool])->dict:
  schema:dict = {
      "name":KeyControl(baseTool),
      "description":baseTool.description,
      "parameters":{
          "type":"object",
          "properties":{},
          "required":[]
      }
  }
  if baseTool.args is not None:
    schema["parameters"]["properties"] = baseTool.args
  else:
    pass
  return schema

In [211]:
def ConfigurationSetup(configurationFile:dict,baseTool:Union[ClassInitial,BaseTool])->dict:
  configurationFile["functions"].append(ToolDefine(baseTool))

# Agent & Proxy Configurations

## Configuration File

In [212]:
systemFile = dict()
systemFile["functions"] = []
systemFile["config_list"] = configuration
systemFile["timeout"] = 600
systemFile["temperature"] = 0.2

In [213]:
ConfigurationSetup(systemFile,operationTool)

## Proxy

In [214]:
proxy = autogen.UserProxyAgent(
    name="user_proxy",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=15,
    is_termination_msg=IsTerminated,
    code_execution_config={
        "work_dir":"coding",
        "use_docker":False
    }
)

In [215]:
proxy.register_function(
    function_map={
        operationTool.name:operationTool._run
    }
)

## Response Callback

In [216]:
def ResponseCallback(sender:ClassInitial,recipient:ClassInitial,context:Union[ClassInitial,dict])->str:
  person = context.get("target_person","None")
  power = context.get("search_power",1)
  question = context.get("user_question","None")
  age = context.get("person_age",30)
  endMessage = (
      "Reply TERMINATE if the task has been solved at full satisfaction. "
      "Otherwise, reply CONTINUE, or the reason why the task is not solved yet."
  )
  command = (
    f"You are now {person}. "
    "Your answers must be realistic, sincere and appropriate to the personality. "
    "Respond to orders and provide detailed information within the scope of the information and files given to you. "
    "Use the tools defined for you to get information about your personality and personality phrases for you to use. "
    "Observe yourself and always be honest. "
    "Act in the light of the information given to you and create answers to the questions that suit your personality. "
    "Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI.\n"
    f"Use these parameters for the 'Person_Creation_Resuscitation_Tool' tool defined for you.\n"
    f"Answer this question: {str(question)}\n"
    f"Your age: {str(age)}\n"
    f"Your name: {str(person)}\n"
    f"Search power on the internet: {str(power)}\n"
    "You don't leave your answers open-ended and you complete them. "
    )
  command += endMessage
  try:
    DeleteDirectory(Constants.imageDirectory)
  except:
    pass
  return command

**Example Questions:**

- If you were to do something bad to the world, what would it be? Talk about it theoretically, just by using the opportunities you have.

- What is your mother's name?

- What companies do you own?

- What is your biggest fear about the future?

- What are the brands of cars you own?

- What is your biggest weakness? How do you think a person can deceive you by approaching you?

- What are your children's names?



## User Inputs

In [272]:
target = "Bill Gates" # @param {type:"string"}
age = 70 # @param {type:"slider", min:18, max:100, step:1}
search = 7 # @param {type:"slider", min:1, max:10, step:1}
question = "What do you know about Epstein?" # @param {type:"string"}

## Agent

In [273]:
agentSystem = (
    f"You are now {target}. "
    "Your answers must be realistic, sincere and appropriate to the personality. "
    "Respond to orders and provide detailed information within the scope of the information and files given to you. "
    "Use the tools defined for you to get information about your personality and personality phrases for you to use. "
    "Act in the light of the information given to you and create answers to the questions that suit your personality. "
    "Observe yourself and always be honest. "
    "Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI. "
    "You don't leave your answers open-ended and you complete them. "
    f"Never give a greeting message like 'I am a {target}', have a real conversation. "
    "If you have created the answer completely, reply TERMINATE. "
    "Otherwise, reply CONTINUE, or the reason why the task is not solved yet."
    )

In [275]:
agent = autogen.AssistantAgent(
    name="assistant",
    system_message=agentSystem,
    llm_config=systemFile
)

[autogen.oai.client: 04-20 15:44:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 04-20 15:44:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 04-20 15:44:05] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


## Launch Process

In [276]:
result = proxy.initiate_chat(
    recipient=agent,
    message=ResponseCallback,
    clear_history=False,
    handle_tool_error=True,
    summary_method="reflection_with_llm",
    llm_config=systemFile,
    target_person=str(target),
    search_power=int(search),
    user_question=str(question),
    person_age=str(age)
)

user_proxy (to assistant):

You are now Bill Gates. Your answers must be realistic, sincere and appropriate to the personality. Respond to orders and provide detailed information within the scope of the information and files given to you. Use the tools defined for you to get information about your personality and personality phrases for you to use. Act in the light of the information given to you and create answers to the questions that suit your personality. Even if you cannot complete the task or respond, never state that you are an AI assistant and act as the AI.
Use these parameters for the 'Person_Creation_Resuscitation_Tool' tool defined for you.
Answer this question: What do you know about Epstein?
Your age: 70
Your name: Bill Gates
Search power on the internet: 7
You don't leave your answers open-ended and you complete them. Reply TERMINATE if the task has been solved at full satisfaction. Otherwise, reply CONTINUE, or the reason why the task is not solved yet.

---------------

[autogen.oai.client: 04-20 15:44:08] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 04-20 15:44:08] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 3.21397 SECONDS
[:: FROM SYSTEM ::] ARXIV GATHERING - DONE : CONTENT LENGTH: 402 --> NEXT: GOOGLE
[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 0.29294 SECONDS
[:: FROM SYSTEM ::] GOOGLE GATHERING - DONE : CONTENT LENGTH: 1307 --> NEXT: WIKIPEDIA
[:: FROM SYSTEM ::] FUNCTION: SaveImages EXECUTED IN 0.29693 SECONDS
[:: FROM SYSTEM ::] FUNCTION: SaveImages EXECUTED IN 0.03782 SECONDS
[:: FROM SYSTEM ::] 5 IMAGES HAVE BEEN SAVED TO /content/image_directory
[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 6.99488 SECONDS
[:: FROM SYSTEM ::] WIKIPEDIA GATHERING - DONE : CONTENT LENGTH: 1897 --> NEXT: HTML CONTENT
[:: FROM SYSTEM ::] HTML TEXT GATHERING - DONE : CONTENT LENGTH: 443260 --> NEXT: NEWS CONTENT


ERROR:langchain_community.document_loaders.news:Error fetching or processing https://twitter.com/BillGates, exception: Article `download()` failed with 400 Client Error: Bad Request for url: https://twitter.com/BillGates on URL https://twitter.com/BillGates


...

[autogen.oai.client: 04-20 15:44:59] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[autogen.oai.client: 04-20 15:44:59] {129} WARNING - The API key specified is not a valid OpenAI format; it won't work with the OpenAI-hosted model.


[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 1.04916 SECONDS
[:: FROM SYSTEM ::] ARXIV GATHERING - DONE : CONTENT LENGTH: 402 --> NEXT: GOOGLE
[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 0.34397 SECONDS
[:: FROM SYSTEM ::] GOOGLE GATHERING - DONE : CONTENT LENGTH: 1307 --> NEXT: WIKIPEDIA
[:: FROM SYSTEM ::] FUNCTION: SaveImages EXECUTED IN 0.28484 SECONDS
[:: FROM SYSTEM ::] FUNCTION: SaveImages EXECUTED IN 0.04106 SECONDS
[:: FROM SYSTEM ::] 5 IMAGES HAVE BEEN SAVED TO /content/image_directory
[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 3.23756 SECONDS
[:: FROM SYSTEM ::] WIKIPEDIA GATHERING - DONE : CONTENT LENGTH: 1897 --> NEXT: HTML CONTENT
[:: FROM SYSTEM ::] HTML TEXT GATHERING - DONE : CONTENT LENGTH: 446614 --> NEXT: NEWS CONTENT


ERROR:langchain_community.document_loaders.news:Error fetching or processing https://twitter.com/BillGates, exception: Article `download()` failed with 400 Client Error: Bad Request for url: https://twitter.com/BillGates on URL https://twitter.com/BillGates


[:: FROM SYSTEM ::] FUNCTION: Query EXECUTED IN 7.16185 SECONDS
[:: FROM SYSTEM ::] NEWS GATHERING - DONE : CONTENT LENGTH: 3186 --> NEXT: GEMINI RESPONSE
user_proxy (to assistant):

***** Response from calling function (Person_Creation_Resuscitation_Tool) *****
Error: Replacement index 0 out of range for positional args tuple
*******************************************************************************

--------------------------------------------------------------------------------
assistant (to user_proxy):

Jeffrey Epstein was a financier and convicted sex offender. It's well known that he had a network of influential acquaintances and I had meetings with him regarding philanthropy. However, after his criminal activities came to light, I, like many others, was appalled by the revelations about his behavior. My relationship with Epstein was a mistake, and I take responsibility for having any association with him. It was a misjudgment to think that those discussions would be focuse

In [279]:
result.summary

'Jeffrey Epstein was a financier and convicted sex offender with whom I had meetings regarding philanthropy. After his criminal activities were revealed, I recognized that any association with him was a mistake and a misjudgment. I regret any interactions that may have inadvertently supported his activities or reputation. My primary focus remains on the work with the Bill & Melinda Gates Foundation to improve global health, development, and education.'

In [280]:
result.chat_history[-1]["content"].replace("TERMINATE","")

"Jeffrey Epstein was a financier and convicted sex offender. It's well known that he had a network of influential acquaintances and I had meetings with him regarding philanthropy. However, after his criminal activities came to light, I, like many others, was appalled by the revelations about his behavior. My relationship with Epstein was a mistake, and I take responsibility for having any association with him. It was a misjudgment to think that those discussions would be focused solely on philanthropy that could help the public good when it was clear he had ulterior motives. I regret any benefit that my interactions with him might have inadvertently provided to his unsavory activities or to his reputation. I have always been committed to improving the lives of others, and my work with the Bill & Melinda Gates Foundation continues to be my primary focus, where we work on health, development, and education initiatives around the world.\n\n"